In [97]:
import pandas as pd
import os
PATH = os.getcwd()
print(PATH)
import glob
import datetime
# to check first rows of files
from csv import reader
# display info options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
from plotly.subplots import make_subplots


c:\_Projects\_REPO\_otherREPO\DataAnalysis


In [98]:
def read_mch_info(file):
    i = 0
    with open(file, 'r') as csv_file:
        csv_reader = reader(csv_file, delimiter=';')
        mch_info = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            mch_info.append(row[0])
            i += 1
            if i == 3:
                break

    return(mch_info)

In [99]:
def concat_files(AllFilesNames):
   # Create an empty list to store the dataframes
    ListDataframe = list()

    # Total file number
    NoFiles = len(AllFilesNames)

    # For each file in the specified directory import the data and add it in the list
    for Filename in AllFilesNames:
        ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

    # Concatenate the files in the list in unique dataframe
    DF_Data = pd.concat(ListDataframe, axis=0, ignore_index=True)

    # DateTime
    DF_Data['DateTime'] = pd.to_datetime(DF_Data['DateTime'], format="%Y-%m-%d %H:%M:%S")
    # ordering ascending
    DF_Data = DF_Data.sort_values(by='DateTime', ascending=True).reset_index(drop=True)

    #Remove duplicates
    DF_Data.drop_duplicates(keep=False, inplace=True)

    return(DF_Data)

In [100]:
# Folder name / TODO: GET FROM USER
foldername = "FCM_B_C/FCM_C/LogExport202107061025/"

In [101]:
# Define files path
Name = "S?*.csv"
# Read all the files name in the directory
DataFiles = glob.glob(foldername + Name)

# Define files path
Name = "A?*.csv"
# Read all the files name in the directory
AlarmFiles = glob.glob(foldername + Name)

# Define files path
Name = "E?*.csv"
# Read all the files name in the directory
EventFiles = glob.glob(foldername + Name)

print(DataFiles)
print(AlarmFiles)
print(EventFiles)

['FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_10__15_44_1.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_10__7_22_45.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_11__0_5_16.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_11__16_47_46.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_11__8_26_31.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_12__17_51_32.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_12__1_9_2.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_12__9_30_17.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_13__10_34_3.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_13__18_55_18.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_13__2_12_47.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_14__11_37_48.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_14__19_59_4.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_14__3_16_33.csv', 'FCM_B_C/FCM_C/LogExport202107061025\\S_2021_6_15__12_41_34.csv', 'FCM_B_C/FCM_C/LogExpo

In [102]:
# Check Machine information
mch_info_check = read_mch_info(DataFiles[0])

for file in (DataFiles + AlarmFiles + EventFiles):
    mch_info = read_mch_info(file)
    if mch_info != mch_info_check:
        print('File ' + file + ' does not correspond to the same machine')
        print(mch_info_check)
        break

print(mch_info_check)

['F60', 'FM 18 M 6996', 'V3.00.09']


In [103]:
LogsStandard = concat_files(DataFiles)
print(LogsStandard.columns)
print(LogsStandard.shape)

LogsAlarms = concat_files(AlarmFiles)
print(LogsAlarms.columns)
print(LogsAlarms.shape)

LogsEvents = concat_files(EventFiles)
print(LogsEvents.columns)
print(LogsEvents.shape)


Index(['DateTime', 'GpsPos', 'PT1', 'PT2', 'TT1', 'TT2', 'TargetTemperature',
       'TemperatureLowLimit', 'TemperatureHighLimit', 'VT', 'TargetViscosity',
       'ViscosityLowLimit', 'ViscosityHighLimit', 'Density', 'FM1_MassFlow',
       'FM1_Density', 'FM1_Temperature', 'FM2_MassFlow', 'FM2_Density',
       'FM2_Temperature', 'FM3_MassFlow', 'FM3_Density', 'FM3_Temperature',
       'FM4_MassFlow', 'FM4_Density', 'FM4_Temperature', 'FT_VolumeFlow',
       'FT_MassFlow', 'FT_Density', 'FT_Temperature', 'SO2', 'CO2', 'SC',
       'CV1_Position', 'CV2_Position', 'CV3_Position', 'CV4_Position',
       'CV5_Position', 'CurrentControl', 'SupplyCurrentPump',
       'CircCurrentPump', 'CurrentFilter', 'F60InAutoMode',
       'ChangeOverInProgress', 'DPT_AI', 'Unnamed: 45'],
      dtype='object')
(259423, 46)
Index(['DateTime', 'AlarmNumber', 'Unnamed: 2'], dtype='object')
(575, 3)
Index(['DateTime', 'GpsPos', 'EventNumber', 'Data', 'Unnamed: 4'], dtype='object')
(409, 5)


In [104]:
# Format LogsStandard
LogsStandard = LogsStandard[['DateTime','GpsPos', 'ChangeOverInProgress','F60InAutoMode','CV1_Position', 'CV2_Position',
                  'CV3_Position', 'CV4_Position', 'CV5_Position', 'SupplyCurrentPump',
       'CircCurrentPump', 'CurrentControl', 'CurrentFilter',
       'PT1', 'PT2', 'TT1', 'TT2', 'TargetTemperature',
       'TemperatureLowLimit', 'TemperatureHighLimit', 'VT', 'TargetViscosity',
       'ViscosityLowLimit', 'ViscosityHighLimit', 'Density', 'FM1_MassFlow',
       'FM1_Density', 'FM1_Temperature', 'FM2_MassFlow', 'FM2_Density',
       'FM2_Temperature', 'FM3_MassFlow', 'FM3_Density', 'FM3_Temperature',
       'FM4_MassFlow', 'FM4_Density', 'FM4_Temperature', 'FT_VolumeFlow',
       'FT_MassFlow', 'FT_Density', 'FT_Temperature', 'SO2', 'CO2', 'SC','DPT_AI']]

# Columns as float
LogsStandard.iloc[:,13:] = LogsStandard.iloc[:,13:].astype(float)
LogsStandard.dtypes

C:\Users\itmolez\AppData\Local\Temp\ipykernel_25156\10817820.py:14: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



DateTime                datetime64[ns]
GpsPos                          object
ChangeOverInProgress             int64
F60InAutoMode                    int64
CV1_Position                     int64
CV2_Position                     int64
CV3_Position                     int64
CV4_Position                     int64
CV5_Position                     int64
SupplyCurrentPump                int64
CircCurrentPump                  int64
CurrentControl                   int64
CurrentFilter                    int64
PT1                            float64
PT2                            float64
TT1                            float64
TT2                            float64
TargetTemperature              float64
TemperatureLowLimit            float64
TemperatureHighLimit           float64
VT                             float64
TargetViscosity                float64
ViscosityLowLimit              float64
ViscosityHighLimit             float64
Density                        float64
FM1_MassFlow             

In [105]:
LogsAlarms = LogsAlarms[['DateTime', 'AlarmNumber']]
LogsAlarms.dtypes

DateTime       datetime64[ns]
AlarmNumber             int64
dtype: object

In [106]:
LogsEvents = LogsEvents[['DateTime', 'GpsPos', 'EventNumber', 'Data']]
LogsEvents.dtypes

DateTime       datetime64[ns]
GpsPos                 object
EventNumber             int64
Data                    int64
dtype: object

# Change Overs

In [107]:
# Identify when Change Over Started and Finished
# ChangeOverInProgress = O : no change
# ChangeOverInProgress = 1 : started
# ChangeOverInProgress = -1 : finished

# Create column with value change
LogsStandard['ChangeoverCMDchange'] = LogsStandard['ChangeOverInProgress'].diff()

# memorize starting DateTimes
COstart = LogsStandard[LogsStandard['ChangeoverCMDchange'] == 1]['DateTime'].tolist()

# memorize finishing DateTimes
COfinish = LogsStandard[(LogsStandard['ChangeoverCMDchange'] == -1)]['DateTime'].tolist()

COs = []
for i in range(len(COfinish)):
    duration = COfinish[i]-COstart[i]
    if duration > datetime.timedelta(minutes = 1):
        COs.append({'Start': COstart[i], 'Finish': COfinish[i], 'Duration': duration})

print('In the logs imported there are ' + str(len(COs)) + ' changeovers')
for CO in COs:
    print('- From ' + str(CO['Start']) + ' to ' + str(CO['Finish']) + '. Duration: ' + str(CO['Duration'])) 


In the logs imported there are 5 changeovers
- From 2021-06-18 20:12:20 to 2021-06-18 21:10:59. Duration: 0 days 00:58:39
- From 2021-06-20 01:43:46 to 2021-06-20 02:30:23. Duration: 0 days 00:46:37
- From 2021-06-27 15:45:54 to 2021-06-27 17:29:40. Duration: 0 days 01:43:46
- From 2021-07-02 04:40:45 to 2021-07-02 05:22:51. Duration: 0 days 00:42:06
- From 2021-07-05 21:18:20 to 2021-07-05 22:13:48. Duration: 0 days 00:55:28


In [108]:
def ChangeOverToDF(CO):
    delta = datetime.timedelta(minutes = 20)
    df = LogsStandard[(LogsStandard['DateTime'] >= CO['Start']-delta) & (LogsStandard['DateTime'] <= CO['Finish']+delta)]
    return(df)

# CHANGE OVER

In [120]:
df = ChangeOverToDF(COs[0])

fig = go.Figure()

# --------------------------------------------------- Valves
greens = ['rgba(0, 128, 0, 0.1)', #green opacity 10%
          'rgba(196, 180, 84, 1)', #vegas gold
          'rgba(64, 224, 208, 1)', #turquoise
          'rgba(69, 75, 27, 1)', #army green
          'rgba(147, 197, 114, 1)', #pistachio
          'rgba(8, 143, 143, 1)', #citrine
        ]

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['ChangeOverInProgress'].astype(bool),
    name="Change Over InProgress",
    fill='tozeroy', mode='none', fillcolor = greens[0],    
    yaxis="y4"
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['CV1_Position'],
    name="CV1 Position",
    line=dict(color = greens[1]), 
    legendgroup="Valves",legendgrouptitle_text="Valves",
    visible='legendonly',
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['CV2_Position'],
    name="CV2 Position",
    line=dict(color = greens[2]),
    legendgroup="Valves",legendgrouptitle_text="Valves",
    visible='legendonly',
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['CV3_Position'],
    name="CV3 Position",
    line=dict(color = greens[3]),    
    legendgroup="Valves",legendgrouptitle_text="Valves",
    visible='legendonly',
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['CV4_Position'],
    name="CV4 Position",
    line=dict(color = greens[4]), 
    legendgroup="Valves",legendgrouptitle_text="Valves",
    visible='legendonly',
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['CV5_Position'],
    name="CV5 Position",
    line=dict(color = greens[5]),
    legendgroup="Valves",legendgrouptitle_text="Valves",
    visible='legendonly',
    yaxis="y3"
))

# ---------------------------------------------------Temperature
reds = ['rgba(255, 87, 51, 1)', #cherry
        'rgba(250, 160, 160, 1)' #pastel red
        ]
if (df['TT1'] != 0).all():
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TT1'],
        name='TT1 - Act Temp',
        line=dict(color = reds[0]),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

if (df['TT2'] != 0).all():
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TT2'],
        name='TT2 - Act Temp',
        line=dict(color = reds[0]),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['TargetTemperature'],
    name='Target Temperature',
    line=dict(color = reds[1]),
    legendgroup="Temperature",legendgrouptitle_text="Temperature",
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['TemperatureLowLimit'],
    name='Temp Low Limit',
    line=dict(color = reds[1], dash='dot'),
    legendgroup="Temperature",legendgrouptitle_text="Temperature",
))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['TemperatureHighLimit'],
    name='Temperature High Limit',
    line=dict(color = reds[1], dash='dot'),
    legendgroup="Temperature",legendgrouptitle_text="Temperature",
))

# --------------------------------------------------- Viscosity
blues = ['rgba(0, 71, 171, 1)', #cobalt blue
        'rgba(70, 130, 180, 1)' #steel blue
        ]

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['VT'],
    name='Viscosity',
    line=dict(color = blues[0]),
    legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
    yaxis="y2"))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['TargetViscosity'],
    name='Target Viscosity',
    line=dict(color = blues[1]),
    legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
    yaxis="y2"))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['ViscosityLowLimit'],
    name='Viscosity Low Limit',
    line=dict(color = blues[1], dash='dot'),
    legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
    yaxis="y2"))

fig.add_trace(go.Scatter(
    x=df['DateTime'],
    y=df['ViscosityHighLimit'],
    name='Viscosity High Limit',
    line=dict(color = blues[1], dash='dot'),
    legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
    yaxis="y2"))

# Create axis objects
fig.update_layout(
    xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

    yaxis=dict(
        title="Temperature"),
    yaxis2=dict(
        title="Viscosity",
        anchor="free",
        overlaying="y",
        side="left",
        position=0
    ),
    yaxis3=dict(
        title="CV Position",
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Change Over",
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),
    legend=dict(orientation="v", x = 1.1)
)

# Update layout properties
fig.update_layout(
    title_text=("ChangeOver Test"),
    legend=dict(groupclick="toggleitem") #avoid grouping all traces
)

config = {'displaylogo': False}
fig.show(config=config)
fig.write_html("ChangeOver_test.html", config=config)


# CONSUMO

# PRESSIONI

# VALVOLE PICCOLO

# RUN & MODES

# Viscosity and Temperature trend

In [ ]:
# Merge Standard logs with Events and Alarms
# Merge all DataFrames
mergeDF2 = pd.merge(Alarms, Events, on='DateTime', how='outer')
mergeDF = pd.merge(DF_Data, mergeDF2, on='DateTime', how='outer')

# Sort again
mergeDF = mergeDF.sort_values(by='DateTime', ascending=True)

# show results
print (mergeDF.shape)
mergeDF.head()

In [ ]:
# Generate limits for intervals of 5 days
mergeDF['DateTime'] = pd.to_datetime(mergeDF['DateTime'])

mindate = mergeDF['DateTime'].min()
maxdate = mergeDF['DateTime'].max()
print(mindate,maxdate)

step = 5

limits = []
limits.append(mindate.date()-datetime.timedelta(days=1))

interval = maxdate - mindate

while interval > datetime.timedelta(days=step):
    limits.append(limits[-1] + datetime.timedelta(days=step))
    interval = interval - datetime.timedelta(days=step)
    
limits.append(maxdate.date()+datetime.timedelta(days=1))

limits = np.array(limits, dtype='datetime64')

# Show Results

for item in limits:
    print (item)

In [ ]:
# Divide all data in slices of 5 days
dfs = []
totalshape = 0

for i in range(len(limits)-1):
    df = pd.DataFrame()
    df = mergeDF[(mergeDF['DateTime'] > limits[i]) & (mergeDF['DateTime'] <= limits[i+1])]
    dfs.append(df)
    
    totalshape = totalshape + df.shape[0]
    print(df.shape)
    
print (totalshape)

In [ ]:
# SPECIFIC FOR THIS DATA SET!!!!!!

# select only the last 6 Dataframes!!
dfs = dfs[-6:]

# show results
for df in dfs:
    print(df.shape)

In [ ]:
# VISCOSITY

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','VT', 'TargetViscosity','ViscosityLowLimit', 'ViscosityHighLimit', 'CV1_Position']]
    df = df.dropna(axis=0, how='all')
    if df.empty:
        continue
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')
    
    fig = go.Figure()
    
    

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Actual Viscosity'
    ))
    
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity'
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        yaxis="y2"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 1]), #compress x axis 10% left

        yaxis=dict(
            title="Viscosity"),
        yaxis2=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Viscosity " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Viscosity_" + mindate + "_" + maxdate + ".html",
                   config=config)


In [ ]:
# TEMPERATURE

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','TT2', 'TargetTemperature','TemperatureLowLimit', 'TemperatureHighLimit', 'CV1_Position']]
    df = df.dropna(axis=0, how='all')
    if df.empty:
        continue
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TT2'],
        name='Actual Temperature'
    ))
    
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature'
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temperature Low Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        yaxis="y2"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 1]), #compress x axis 10% left

        yaxis=dict(
            title="Temperature"),
        yaxis2=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Temperature " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Temperature_" + mindate + "_" + maxdate + ".html",
                   config=config)

In [ ]:
# Flow, Alarms & Events

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','FT_VolumeFlow', 'FT_MassFlow', 'AlarmNumber', 'EventNumber']]
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')

    fig = go.Figure()
    
    dfGraph = df[['DateTime','FT_VolumeFlow']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['FT_VolumeFlow'],
        name='Flow Transmitter VolumeFlow'
    ))
    
    if df['FT_MassFlow'].mean() != 0:
        dfGraph = df[['DateTime','FT_MassFlow']]
        dfGraph = dfGraph.dropna(axis=0, how='any')

        fig.add_trace(go.Scatter(
            x=dfGraph['DateTime'],
            y=dfGraph['FT_MassFlow'],
            name='Flow Transmitter MassFlow',
            yaxis="y2"
        ))


    dfGraph = df[['DateTime','AlarmNumber']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['AlarmNumber'].astype(str),
        name='Alarm Number',
        mode='markers',marker_symbol='x',
        marker_color="red", marker_size=8,
        yaxis="y3"
    ))

    dfGraph = df[['DateTime','EventNumber']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['EventNumber'].astype(str),
        name='Event Number',
        mode='markers', marker_symbol='star',
        marker_color="midnightblue",
        marker_size=8,
        yaxis="y4"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

        yaxis=dict(
            title="Volume Flow"),
        yaxis2=dict(
            title="Mass Flow",
            anchor="free",
            overlaying="y",
            side="left",
            position=0
        ),
        yaxis3=dict(
            title="Alarm",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        yaxis4=dict(
            title="Event",
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Flow, Alarms & Events " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Flow-Alarms-Events_" + mindate + "_" + maxdate + ".html",
                   config=config)

In [ ]:
import tkinter
import customtkinter  # <- import the CustomTkinter module

root_tk = tkinter.Tk()  # create the Tk window like you normally do
root_tk.geometry("400x240")
root_tk.title("CustomTkinter Test")

def button_function():
    print("button pressed")

# Use CTkButton instead of tkinter Button
button = customtkinter.CTkButton(master=root_tk, corner_radius=10, command=button_function)
button.place(relx=0.5, rely=0.5, anchor=tkinter.CENTER)

root_tk.mainloop()

In [ ]:
# OPEN A FOLDER
import os
path = PATH
path = os.path.realpath(path)
os.startfile(path)